In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import datetime
import os
import skimage
from skimage.transform import resize
import random
from tensorflow.python.keras.utils.data_utils import Sequence
from scipy.ndimage import zoom
from scipy.ndimage import shift
from keras.applications import Xception

In [22]:
#input variables
target_path = "C:\\Users\\awe2\\DL_DES-master\\data\\sdss-galaxyzoo\\high_certainty\\sdss_metadata\\"

#path = 'image_arrays_new_new\\'
#validation_path = path + 'validation'
#training_path = path + 'training'
#test_path = path + 'test'

#model variables
batch_size = 32 #
learning_rate = 1e-4 

params = {'dim': (120,120),
          'batch_size': batch_size,
          'n_classes': 2,
          'n_channels': 3,
          'shuffle': True}

#more parameters means more prone to overfitting, and I am 5/3 times worse on parameters compared to the paper I have
#based this on. (5 bands instead of 3) I need to find ways to add more regularization, or otherwise might try reducing my number
#of layers to reduce the number of parameters.

In [23]:
#https://stackoverflow.com/questions/37119071/scipy-rotate-and-zoom-an-image-without-changing-its-dimensions/48097478
def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

In [24]:
#https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
class DataGenerator(Sequence):

    def __init__(self, list_IDs, labels, batch_size=32, dim=(120,120), n_channels=3,
                 n_classes=2, shuffle=True):
     #   'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def on_epoch_end(self):
    #'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
    #'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    # Initialization

        X = np.zeros((self.batch_size, *self.dim, self.n_channels), dtype=int)
        
        X_resized = np.zeros((self.batch_size, 224, 224, self.n_channels), dtype=int)
        #X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
      
      # Generate data and perform augmentation
        for i, ID in enumerate(list_IDs_temp):
            
          # Store sample
            X[i,:,:,:] = np.load('HP_inputs/' + ID + '.npy')[:,:,1:4]
            #X[i,4:68,4:68,:] = np.load('image_arrays/' + ID + '.npy')[:,:,1:4] 
            
            #the arrays are saved as u,g,i,r,z . i want to provide it in order of i,r,g
            i_filter = X[i,:,:,0]
            r_filter = X[i,:,:,1]
            g_filter = X[i,:,:,2]
            
            X[i,:,:,0] = i_filter
            X[i,:,:,1] = r_filter
            X[i,:,:,2] = g_filter
            #resize
            X_resized[i,:,:,:] = (255 * resize(X[i,:,:,:], (224,224,3))).astype(np.uint8)
            
            #flip
            if random.random() > 0.5:
                X_resized[i,] = np.flip(X_resized[i,],0)
            if random.random() > 0.5:
                X_resized[i,] = np.flip(X_resized[i,],1)
            
            #shift
            if random.random() > 0.5 :
                X_resized[i,] = shift(X_resized[i,], (4,0,0), mode='nearest')
            elif random.random() > 0.5 :
                X_resized[i,] = shift(X_resized[i,], (-4,0,0), mode='nearest')
                              
            if random.random() > 0.5 :
                X_resized[i,] = shift(X_resized[i,], (0,4,0), mode='nearest')
            elif random.random() > 0.5 :
                X_resized[i,] = shift(X_resized[i,], (0,-4,0), mode='nearest')
          
            #zoom in/out
            zoom_factor = random.uniform(0.75,1.3)
            X_resized[i,] = clipped_zoom(X_resized[i,],zoom_factor)
            
            #rotate
            angle = 45*random.random()
            X_resized[i,] = skimage.transform.rotate(X_resized[i,], angle=angle, mode='reflect')
            
            # Store class
            y[i] = self.labels[ID]
    
        if self.n_classes > 2:
            return X_resized, keras.utils.to_categorical(y, num_classes=self.n_classes)
        else:
            return X_resized, y

    def __len__(self):
    #'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
    #  'Generate one batch of data'
      # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

      # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

      # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

In [25]:
#generate the targets.
a = []
for file in (os.listdir(target_path)):
    target = pd.read_csv(target_path+file,usecols=[14])
    a.append((target["target"].values))
targets = np.concatenate([a[0],a[1],a[2],a[3],a[4],a[5],a[6],a[7],a[8]])

In [26]:
with open("labels.txt","r") as inf:
    labels = eval(inf.read())

In [27]:
#now we shuffle our data list, and partition into train, test, and validation sets.
data_list = os.listdir("HP_inputs")
random.shuffle(data_list)
#we want 20% of the data reserved for testing, and then another 15% reserved for validation.
top_index = len(data_list)
first_index = int(round(top_index * 0.65))
second_index = int(round(top_index * 0.15)) + first_index

train_list = data_list[0:first_index]
val_list = data_list[first_index:second_index]
test_list = data_list[second_index::]

for i,file in enumerate(train_list):
    train_list[i] = file.split('.')[0]
for i,file in enumerate(val_list):
    val_list[i] = file.split('.')[0]

partition = {'train':train_list,'validation':val_list}

In [28]:
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

In [29]:
#so this is pretty neat, you can create a keras callback to display on tensorboard using a simplified summary tf api

#and also this is an example of how to change the lr on the fly, which is pretty handy
#https://keras.io/callbacks/


"""
    file_writer = tf.summary.create_file_writer(logdir + "/metrics")
    file_writer.set_as_default()
"""
def lr_schedule(epoch,lr):

#Returns a custom learning rate that decreases as epochs progress.
    if epoch > 5:
        lr = 1e-5
    if epoch > 10:
        lr = 1e-6

    tf.summary.scalar('learning_rate', tensor=lr)
    return lr

lr_callback = keras.callbacks.LearningRateScheduler(lr_schedule)

#logdir="summaries/scalars/" + str(datetime.datetime.now().timestamp())
#tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir,
#                                                   histogram_freq=1,
#                                                   write_graph=False,
#                                                   write_grads=True,)
#                                                    write_images=True)
#will it still print stuff

In [30]:
"""
base_model = Xception(input_shape=(72,72,3), weights='imagenet', include_top=False)
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.7)(x)
x = keras.layers.Dense(1024, activation="relu", name='second_last_layer')(x)
predictions = keras.layers.Dense(1, activation="sigmoid")(x)

model_final = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model_final.compile(loss = "binary_crossentropy", optimizer = optimizers.Adam(lr=learning_rate), metrics=["accuracy"])
"""

'\nbase_model = Xception(input_shape=(72,72,3), weights=\'imagenet\', include_top=False)\nx = base_model.output\nx = keras.layers.GlobalAveragePooling2D()(x)\nx = keras.layers.Dense(1024, activation=\'relu\')(x)\nx = keras.layers.Dropout(0.7)(x)\nx = keras.layers.Dense(1024, activation="relu", name=\'second_last_layer\')(x)\npredictions = keras.layers.Dense(1, activation="sigmoid")(x)\n\nmodel_final = Model(inputs=base_model.input, outputs=predictions)\n\nfor layer in base_model.layers:\n    layer.trainable = False\n\nmodel_final.compile(loss = "binary_crossentropy", optimizer = optimizers.Adam(lr=learning_rate), metrics=["accuracy"])\n'

In [31]:
steps_to_take = int(len(train_list)/batch_size)
val_steps_to_take = int(len(val_list)/batch_size)
                #typically be equal to the number of unique samples if your dataset
                #divided by the batch size.

print(steps_to_take)
print(val_steps_to_take)

785
181


In [32]:
keras.backend.clear_session()
tf.reset_default_graph()

In [33]:
#lets try it how we ran VGG16...
"""
Input_layer = layers.Input(shape=(32,32,3))
base_model = vgg16.VGG16(include_top=False, weights='imagenet',input_tensor=Input_layer)

x=base_model.output
x=layers.GlobalAveragePooling2D()(x)
x=layers.Dropout(dropout)(x)
x=layers.Dense(1024,activation='relu')(x)
"""
Input_layer = keras.layers.Input(shape=(224,224,3))
base_model = keras.applications.Xception(input_tensor=Input_layer,weights='imagenet',include_top=False)

x= base_model.output
x= keras.layers.GlobalAveragePooling2D()(x)
x= keras.layers.Dense(1024, activation=tf.nn.relu)(x)
x= keras.layers.Dropout(0.7)(x)
x= keras.layers.Dense(128, activation=tf.nn.relu)(x)
preds= keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)

W0722 08:58:10.368887  5632 nn_ops.py:4224] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [34]:
from tensorflow.keras import Model
model=Model(inputs=base_model.input,outputs=preds)

In [35]:
#for i,layer in enumerate(model.layers):
#    print(i,layer.name)

In [36]:
#need to train some of the later layers it seems; not just my dense. 
#Huerta and kahn use 41, they have access to bluewaters tho...

for layer in base_model.layers:
    layer.trainable = False
    
    


In [37]:
#nope this doesnt work
"""
keras.backend.clear_session()
tf.reset_default_graph()

base_model = keras.applications.Xception(input_shape=(72,72,3),weights='imagenet',include_top=False)
#base_model = keras.applications.Xception(weights='imagenet',include_top=False)
for layer in base_model.layers:
    layer.trainable = False

model = keras.Sequential([])
model.add(keras.layers.InputLayer((72,72,3), name='input'))
model.add(base_model)
model.add(keras.layers.GlobalAveragePooling2D(name='global_after_transfer'))
model.add(keras.layers.Dense(1024, activation=tf.nn.relu,name='first_dense'))
model.add(keras.layers.Dropout(0.7))
model.add(keras.layers.Dense(1024, activation=tf.nn.relu,name='second_dense'))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid,name='predictor'))

model.compile(loss=keras.losses.binary_crossentropy, optimizer = keras.optimizers.Adam(lr=learning_rate), metrics=["accuracy"])
"""

'\nkeras.backend.clear_session()\ntf.reset_default_graph()\n\nbase_model = keras.applications.Xception(input_shape=(72,72,3),weights=\'imagenet\',include_top=False)\n#base_model = keras.applications.Xception(weights=\'imagenet\',include_top=False)\nfor layer in base_model.layers:\n    layer.trainable = False\n\nmodel = keras.Sequential([])\nmodel.add(keras.layers.InputLayer((72,72,3), name=\'input\'))\nmodel.add(base_model)\nmodel.add(keras.layers.GlobalAveragePooling2D(name=\'global_after_transfer\'))\nmodel.add(keras.layers.Dense(1024, activation=tf.nn.relu,name=\'first_dense\'))\nmodel.add(keras.layers.Dropout(0.7))\nmodel.add(keras.layers.Dense(1024, activation=tf.nn.relu,name=\'second_dense\'))\nmodel.add(keras.layers.Dense(1, activation=tf.nn.sigmoid,name=\'predictor\'))\n\nmodel.compile(loss=keras.losses.binary_crossentropy, optimizer = keras.optimizers.Adam(lr=learning_rate), metrics=["accuracy"])\n'

In [38]:
#define an optimizer, loss, and accuracy metric.
adam = tf.keras.optimizers.Adam(1e-5)
model.compile(optimizer=adam, loss=keras.losses.binary_crossentropy,metrics=['accuracy'])

In [39]:
filepath = "models//Xception-khan-0.h5"
ModelCheckpointCB = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

In [40]:
hist = model.fit_generator(generator=training_generator,
                    steps_per_epoch=steps_to_take, 
                    epochs=5,
                    validation_data=validation_generator,
                    validation_steps=val_steps_to_take,
                    verbose=1,
                    callbacks=[ModelCheckpointCB,lr_callback])

C:\Users\awe2\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\awe2\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Epoch 1/5


C:\Users\awe2\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


784/785 [============================>.] - ETA: 7s - loss: 0.6966 - acc: 0.4968 
Epoch 00001: val_loss improved from inf to 0.69315, saving model to models//Xception-khan-0.h5
785/785 [==============================] - 7607s 10s/step - loss: 0.6966 - acc: 0.4966 - val_loss: 0.6931 - val_acc: 0.4974
Epoch 2/5
784/785 [============================>.] - ETA: 7s - loss: 0.6949 - acc: 0.5003 
Epoch 00002: val_loss did not improve from 0.69315
785/785 [==============================] - 7004s 9s/step - loss: 0.6948 - acc: 0.5004 - val_loss: 0.6932 - val_acc: 0.4979
Epoch 3/5
784/785 [============================>.] - ETA: 7s - loss: 0.6945 - acc: 0.5014 
Epoch 00003: val_loss did not improve from 0.69315
785/785 [==============================] - 6987s 9s/step - loss: 0.6945 - acc: 0.5014 - val_loss: 0.6932 - val_acc: 0.4979
Epoch 4/5
 40/785 [>.............................] - ETA: 1:05:04 - loss: 0.6944 - acc: 0.4930

KeyboardInterrupt: 

In [41]:
for layer in model.layers[:41]:
    layer.trainable=False
for layer in model.layers[41:]:
    layer.trainable=True
    
filepath = "models//Xception-khan-1.h5"
ModelCheckpointCB = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

adam = tf.keras.optimizers.Adam(1e-6)
model.compile(optimizer=adam, loss=keras.losses.binary_crossentropy,metrics=['accuracy'])

In [ ]:
hist2 = model.fit_generator(generator=training_generator,
                    steps_per_epoch=steps_to_take, 
                    epochs=5,
                    validation_data=validation_generator,
                    validation_steps=val_steps_to_take,
                    verbose=1,
                    callbacks=[ModelCheckpointCB,lr_callback])

Epoch 1/5


In [ ]:
#test_loss, test_acc = model.evaluate(test_images, test_target)
#print('Test accuracy:', test_acc)
#print('Test loss:', test_loss)

In [ ]:
#y_prob = model.predict(X)

In [ ]:
#source list
"""
https://fizzylogic.nl/2017/05/08/monitor-progress-of-your-keras-based-neural-network-using-tensorboard/

https://stackoverflow.com/questions/41032551/how-to-compute-receiving-operating-characteristic-roc-and-auc-in-keras

https://arxiv.org/pdf/1711.05744.pdf

https://arxiv.org/pdf/1807.00807.pdf

https://github.com/jameslawlor/kaggle_galaxy_zoo/blob/master/galaxy_zoo_keras.ipynb

https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

#https://stackoverflow.com/questions/37119071/scipy-rotate-and-zoom-an-image-without-changing-its-dimensions/48097478

https://distill.pub/2018/building-blocks/ what I want to do with this after it is working.

https://github.com/khanx169/DL_DES/blob/master/deeplearning/Xception_final.ipynb
"""

In [ ]:
"""
training just the top layers; 1 epoch like Khan

784/785 [============================>.] - ETA: 2s - loss: 0.3144 - acc: 0.8830
Epoch 00001: val_loss improved from inf to 15.60131, saving model to models//Xception-khan-0.h5
785/785 [==============================] - 2440s 3s/step - loss: 0.3143 - acc: 0.8831 - val_loss: 15.6013 - val_acc: 0.5312

And then training all layers after 41:

Epoch 1/5
784/785 [============================>.] - ETA: 1s - loss: 0.0695 - acc: 0.9696
Epoch 00001: val_loss improved from inf to 59.03588, saving model to models//Xception-khan-1.h5
785/785 [==============================] - 1865s 2s/step - loss: 0.0696 - acc: 0.9695 - val_loss: 59.0359 - val_acc: 0.4722
Epoch 2/5
784/785 [============================>.] - ETA: 1s - loss: 0.0417 - acc: 0.9805
Epoch 00002: val_loss did not improve from 59.03588
785/785 [==============================] - 1841s 2s/step - loss: 0.0417 - acc: 0.9806 - val_loss: 140.0926 - val_acc: 0.5432
Epoch 3/5
784/785 [============================>.] - ETA: 1s - loss: 0.0381 - acc: 0.9814
Epoch 00003: val_loss did not improve from 59.03588
785/785 [==============================] - 1839s 2s/step - loss: 0.0381 - acc: 0.9814 - val_loss: 160.0299 - val_acc: 0.5516
Epoch 4/5
784/785 [============================>.] - ETA: 1s - loss: 0.0353 - acc: 0.9823
Epoch 00004: val_loss did not improve from 59.03588
785/785 [==============================] - 1835s 2s/step - loss: 0.0354 - acc: 0.9822 - val_loss: 215.8511 - val_acc: 0.5551
Epoch 5/5
784/785 [============================>.] - ETA: 1s - loss: 0.0348 - acc: 0.9823
Epoch 00005: val_loss did not improve from 59.03588
785/785 [==============================] - 1840s 2s/step - loss: 0.0349 - acc: 0.9823 - val_loss: 193.4825 - val_acc: 0.5447

#training all layers after the input:

RESOURCE ERROR
"""